In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Vincent\\Desktop\\Weather-Forecast-Trials\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Vincent\\Desktop\\Weather-Forecast-Trials'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_file_path: Path
    train_array_path: Path
    test_array_path: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
## Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        create_directories([self.config.output_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            trained_model_file_path=config.trained_model_file_path,
            train_array_path=config.train_array_path,
            test_array_path=config.test_array_path
        )

        return model_trainer_config

In [8]:
## 5. Update the components

import numpy as np
import os
import sys
from dataclasses import dataclass
from sklearn.metrics import confusion_matrix
from src.exception import CustomException
from src import logger
from src.utils.common import save_object,evaluate_models
from src.params import params
from src.models import models
import warnings
warnings.filterwarnings("ignore")

In [9]:
## 5. Update the components

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config


    def initiate_model_trainer(self):
        try:
            train_array=np.load(self.config.train_array_path)
            test_array=np.load(self.config.test_array_path)
            logger.info("Split training and test input data")
            X_train,y_train,X_test,y_test=(
                train_array[:,:-1],
                train_array[:,-1],
                test_array[:,:-1],
                test_array[:,-1]
            )

            model_report:dict=evaluate_models(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,
                                             models=models(),param=params())
            
            ## To get best model score from dict
            best_model_score = max(sorted(model_report.values()))

            ## To get best model name from dict
            best_model_name = list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]
            best_model = models()[best_model_name]

            if best_model_score<0.6:
                raise CustomException("No best model found")
            
            logger.info(f"Best found model on both training and testing dataset")

            save_object(
                file_path=self.config.trained_model_file_path,
                obj=best_model
            )

            best_model = best_model.fit(X_train, y_train)

            predicted=best_model.predict(X_test)

            cm = confusion_matrix(y_test, predicted)
            
            logger.info(f"best model: {best_model_name}; best F1 score: {best_model_score}; confusion matrix: {cm};")
            
            return best_model_name, best_model_score, cm
            

        except Exception as e:
            raise CustomException(e,sys)

In [10]:
## 6. Update the pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.initiate_model_trainer()
except Exception as e:
  raise e

[2024-07-11 21:14:43,233: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 21:14:43,233: INFO: common: created directory at: output]
[2024-07-11 21:14:43,233: INFO: common: created directory at: output/model_trainer]
[[-0.24647819  0.23498784 -0.27536409 ...  0.          0.
   0.        ]
 [-0.96619454 -1.14146495 -0.08597762 ...  0.21505166  0.88357648
   0.        ]
 [ 1.94396291  2.55248488 -0.27536409 ...  0.43010333  1.76715297
   0.        ]
 ...
 [ 1.69362678  1.52716801 -0.27536409 ...  1.72041332  1.76715297
   0.        ]
 [ 0.39500813  1.00748685 -0.27536409 ...  2.79567164  0.88357648
   0.        ]
 [ 0.55146821  0.44566939 -0.27536409 ...  1.93546498  0.
   0.        ]]
[[ 1.19295453  0.65635094 -0.27536409 ...  0.21505166  1.76715297
   1.        ]
 [ 0.98955643  0.10857891 -0.18067086 ...  1.29030999  1.76715297
   0.        ]
 [-0.43423028 -0.28469332 -0.27536409 ...  1.93546498  0.
   0.        ]
 ...
 [-0.46552229 -0.76223816  0.17442877 .